In [6]:
import numpy as np
import pandas as pd
import os
import dotenv
import psycopg
from sqlalchemy import create_engine

import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

import plotly.express as px
import plotly.figure_factory as ff

In [7]:
dotenv.load_dotenv()
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')

dbms = 'postgresql'
package = 'psycopg'
user = 'postgres'
password = POSTGRES_PASSWORD
host = 'localhost'
port = '5432'
db = 'contrans'

engine = create_engine(f'{dbms}+{package}://{user}:{password}@{host}:{port}/{db}')

In [3]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Create features that we need, but won't change depending on what the user does
myquery = '''
SELECT *
FROM members
'''
data = pd.read_sql(myquery, con=engine)

In [11]:
data2 = data[['full_name', 'state_abbrev', 'district_code', 'party']]
data2['party'] = data2['party'].str[0:1]
display_name = [n + ' (' + p + ', ' + s + '-' + str(d) + ')'
                for n, s, d, p in zip(data2['full_name'], data2['state_abbrev'], data2['district_code'], data2['party'])]
display_name = [x.replace('-0','') for x in display_name]
display_name

['Robert B. Aderholt (R, AL-4)',
 'Jake Auchincloss (D, MA-4)',
 'Mark E. Amodei (R, NV-2)',
 'Alma S. Adams (D, NC-12)',
 'Pete Aguilar (D, CA-33)',
 'Rick W. Allen (R, GA-12)',
 'Jodey C. Arrington (R, TX-19)',
 'Mark Alford (R, MO-4)',
 'Gabe Amo (D, RI-1)',
 'Yassamin Ansari (D, AZ-3)',
 'Angela D. Alsobrooks (D, MD)',
 'Sanford D. Bishop, Jr. (D, GA-2)',
 'Cliff Bentz (R, OR-2)',
 'Stephanie I. Bice (R, OK-5)',
 'Lauren Boebert (R, CO-4)',
 'Tammy Baldwin (D, WI)',
 'John Boozman (R, AR)',
 'Marsha Blackburn (R, TN)',
 'Gus M. Bilirakis (R, FL-12)',
 'Vern Buchanan (R, FL-16)',
 'John Barrasso (R, WY)',
 'Michael F. Bennet (D, CO)',
 'Richard Blumenthal (D, CT)',
 'Suzanne Bonamici (D, OR-1)',
 'Joyce Beatty (D, OH-3)',
 'Andy Barr (R, KY-6)',
 'Julia Brownley (D, CA-26)',
 'Ami Bera (D, CA-6)',
 'Cory A. Booker (D, NJ)',
 'Brian Babin (R, TX-36)',
 'Donald S. Beyer, Jr. (D, VA-8)',
 'Mike Bost (R, IL-12)',
 'Brendan F. Boyle (D, PA-2)',
 'Don Bacon (R, NE-2)',
 'Jim Banks (R, IN)

In [23]:
b = 'A000055'

myquery = f'''
SELECT *
FROM members
WHERE bioguide_id = '{b}'
'''

member_info = pd.read_sql(myquery, con=engine)
member_info = member_info.drop(['bioguide_id', 'image', 'fec_id', 'bioname', 'icpsr'], axis=1)

ff.create_table(member_info.T.reset_index().rename({'index':'', 0:''}, axis=1))

In [4]:
myquery = f'''
SELECT image
FROM members
WHERE bioguide_id = 'A000055'
'''
image_df = pd.read_sql(myquery, con=engine)

image_df['image'][0]

'https://www.congress.gov/img/member/a000055_200.jpg'

In [13]:
b = 'A000055'

myquery = f'''
SELECT c.comparison_member, 
       c.agree, 
       m.left_right_ideology,
       m.party
FROM members m
INNER JOIN (
    SELECT vc.comparison_member,
           vc.agree
    FROM members m
    INNER JOIN vote_compare vc
        ON m.bioname = vc.bioname
    WHERE m.bioguide_id = '{b}'
) c
    ON m.bioname = c.comparison_member
'''

vote_data = pd.read_sql(myquery, con=engine)
vote_data


,comparison_member,agree,left_right_ideology,party
0,"DIAZ-BALART, Mario",0.928826,0.281,Republican
1,"SCOTT, David",0.345196,-0.304,Democrat
2,"CASE, Ed",0.355872,-0.205,Democrat
3,"TURNER, Michael R.",0.900356,0.265,Republican
4,"COLE, Tom",0.950178,0.323,Republican
...,...,...,...,...
437,"BARRETT, Tom",0.921708,0.509,Republican
438,"BAUMGARTNER, Michael",0.935943,0.489,Republican
439,"BEGICH, Nicholas J., III",0.896797,0.634,Republican
440,"BELL, Wesley",0.334520,-0.567,Democrat


In [16]:
import plotly.express as px
fig = px.scatter(vote_data,
                 x = 'left_right_ideology',
                 y = 'agree',
                 hover_name = 'comparison_member',
                 color = 'party',
                 color_discrete_map={'Democrat':'blue', 'Republican':'red'})
fig.show()